# Root Finding to solve the inverse kinematic
When we want to find a solution for the inverse kinematic we need to find a solution of a set of functions with multiple variables. We can use the Newton method to find the root of a function. In Exercise 2, we learned how to find the root of a function with one variable: 

### From previous Exercise: Newton Method with one variable

To implement the Newton method, we used the first Taylor approximation (tangent) at the point $x_n$:
$$f(x) = f(x_n) + f'(x_n)(x-x_n)$$
We set $f(x_{n+1}) = 0$ and solved for $x$:
$$0 = f(x_n) + f'(x_n)(x_{n+1}-x_n)$$
$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$$  
The calculation rule for the next x which is closer to the root is thus iteratively called over and over again. The calculation rule according to the Newton method is therefore:
$$x_0 = start value$$
$$x_{1} = x_0 - \frac{f(x_0)}{f'(x_0)}$$
$$x_{2} = x_{1} - \frac{f(x_1)}{f'(x_{1})}$$
...
$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$$ 
We do this until we have reached the desired accuracy, i.e., the desired distance of $f(x_{n+1})$ to 0.

**Summary**: The Newton method is an iterative method for finding the root of a function. The calculation rule for the next x which is closer to the root is thus iteratively called over and over again. When look at the calculation rule of the Newton method:
$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$$
We see that we need the function $f(x)$ and its derivative $f'(x)$. 

### Root finding with multiple variables
What do we want to solve for if we say that we want to find the root of a function with multiple variables? In the lecture we learned that if we solve an equation with two unkown variables like: 
$$
  f_1(x,y) = c_1,
$$
we get a 1-D curve in a 2-D space. We can also call this equation a constraint. One example of this are contour plots. We can plot a function with two variables in a 3-D plot. Therefore we estimate a value $z$ for the values $x$ and $y$ with $z=f(x,y)$. In a contour plot we plot a contour line for a specific value of $z$ (or in the equation from above for $c_1$). 

Let's look at an example of a contour plot for the function $f(x,y) = 0.5*sin(x) + cos(y)$:

In [ ]:
import Pkg
Pkg.activate("../Exercise 4/OpenMEnv")
Pkg.add("Revise")
Pkg.add("GLMakie")
Pkg.add("ForwardDiff")
Pkg.add("LinearAlgebra")
Pkg.add("FiniteDifferences")
using Revise
include("../OpenManipulatorLib/OpenManipulatorKinematics.jl")
import .OpenManipulatorKinematics

In [ ]:
using GLMakie

function contourplot(x, y, z)
    GLMakie.activate!()
    with_theme(theme_dark()) do
        fig =  Figure(resolution = (1200,800))
        axs = [Axis3(fig[1,i]; aspect = :data) for i in 1:2]
        surface!(axs[1], x, y, z; colormap=:viridis, colorrange=(minimum(z), maximum(z)),
            transparency=false)
        contour!(axs[1], x, y, z; levels=20, colormap=:viridis, linewidth=2,
            colorrange=(minimum(z), maximum(z)), transformation=(:xy, minimum(z)),
            transparency=true)
        contour3d!(axs[2], x, y, z; levels = 20, transparency = true)
        hidedecorations!.(axs; grid = false)
        fig
    end
end

In [ ]:
# let's define a function
f(x,y) = 0.5*sin(x) + cos(y)
x = LinRange(-2π, 2π, 1000)
y = LinRange(-2π, 2π, 1000)
z = [f(x[i], y[j]) for i in eachindex(x), j in eachindex(y)]

# and plot it
contourplot(x, y, z)

The surfaceplot on the left shows the function $f(x,y) = 0.5*sin(x) + cos(y)$ in a 3-D plot with a contourplot in the x,y-plane. On the right is a 3-D contourplot. Each color in the contourplot on the right represents a different value of $z= f(x,y) = 0.5*sin(x) + cos(y)$. So we can interpret it as as a visulaization of where the function $f(x,y) = 0.5*sin(x) + cos(y)$ is equal to a specific value like $f(x,y) = 0.5$. 

If we add another constraint like:
$$
  f_2(x,y) = c_2,
$$
we have two constraints. We get two 1-D curves in a 2-D space. The intersection of these two curves is the solution to the equation (a 0-D result). This point is the solution to the equation system. In general: Each independant equation reduces the dimension of the solution space by one. 

For example, if we solve the equation system:
$$
  f_1(x,y) = 0.5*sin(x) + cos(y) = c_1 = 0,
$$
$$
  f_2(x,y) = x = c_2 = 0,
$$
we get the point (or points - since we can also have multiple solutions), where the two curves of each solution intersect. Here we get all points where $x=0$ and $0.5*sin(0) + cos(y) = 0$. One solution therefore ist $x=0, y=\frac{\pi}{2}, z=0$. 

We can rewrite this system of equations into a vector form and make it more general: 

## The Root Finding Problem

We want to find a vector $\mathbf{x} = (x_1, \dots, x_n)$, given on a continuous vector-valued function $\mathbf{f}= (f_1, \dots f_n)$ mapping from $\mathbb{R}^n$ into $\mathbb{R}^n$, such that
$$
\begin{split}\begin{split}
  f_1(x_1,\dots,x_n) &= 0,\\
  f_2(x_1,\dots,x_n) &= 0,\\
  &\vdots\\
  f_n(x_1,\dots,x_n) &= 0.
\end{split}\end{split}
$$
We can also write this in a more compact form:
$$
  \mathbf{f}(\mathbf{x}) = \mathbf{0}.
$$
The solution $\mathbf{x}$ is called the **root** of the vector-valued function. How can we find the root of a function with multiple variables? 

## Newton Method with multiple variables

**Linear Model**: We can use the first Taylor approximation (tangent) at the point $\mathbf{x}_n$ and use the **Newton method** to find the root of the function with multiple variables:
$$
\mathbf{f}(\mathbf{x}+\mathbf{h}) = \mathbf{f}(\mathbf{x}) + \mathbf{J}(\mathbf{x})\mathbf{h} + O(\| \mathbf{h} \|^2),
$$
where $\mathbf{J}(\mathbf{x})$ is the Jacobian matrix of $\mathbf{f}$ at $\mathbf{x}$:
$$
\begin{split}\mathbf{J}(\mathbf{x}) =
  \begin{bmatrix}
    \rule[2mm]{0pt}{1em}\frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2} & \cdots & \frac{\partial f_1}{\partial x_n}\\[2mm]
    \frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2} & \cdots & \frac{\partial f_2}{\partial x_n}\\[1mm]
    \vdots & \vdots & & \vdots\\[1mm]
    \rule[-3mm]{0pt}{1em} \frac{\partial f_n}{\partial x_1} & \frac{\partial f_n}{\partial x_2} & \cdots & \frac{\partial f_n}{\partial x_n}
  \end{bmatrix} = \left[ \frac{\partial f_i}{\partial x_j} \right]_{\,i,j=1,\ldots,n}.\end{split}
$$

We set $\mathbf{f}(\mathbf{x}_{n+1}) = \mathbf{0}$ and solved for $\mathbf{x}$:
$$
\begin{split}\begin{split}
  \mathbf{0} &= \mathbf{f}(\mathbf{x}_n) + \mathbf{J}(\mathbf{x}_n)\mathbf{h}\\
  \mathbf{h} &= -\mathbf{J}(\mathbf{x}_n)^{-1}\mathbf{f}(\mathbf{x}_n)\\
  \mathbf{x}_{n+1} &= \mathbf{x}_n - \mathbf{J}(\mathbf{x}_n)^{-1}\mathbf{f}(\mathbf{x}_n).
\end{split}\end{split}
$$
The calculation rule for the next $\mathbf{x}$ which is closer to the root is thus iteratively called over and over again. The calculation rule according to the Newton method is therefore:
$$
\mathbf{x}_0 = \text{start value}
$$
$$
\mathbf{x}_{1} = \mathbf{x}_0 - \mathbf{J}(\mathbf{x}_0)^{-1}\mathbf{f}(\mathbf{x}_0)
$$
$$
\mathbf{x}_{2} = \mathbf{x}_{1} - \mathbf{J}(\mathbf{x}_{1})^{-1}\mathbf{f}(\mathbf{x}_{1})
$$
$$
...
$$
$$
\mathbf{x}_{n+1} = \mathbf{x}_n - \mathbf{J}(\mathbf{x}_n)^{-1}\mathbf{f}(\mathbf{x}_n)
$$
We do this until we have reached the desired accuracy, i.e., the desired distance of $\mathbf{f}(\mathbf{x}_{n+1})$ to $\mathbf{0}$.

**Summary**: The Newton method is an iterative method for finding the root of a function with multiple variables. The calculation rule for the next $\mathbf{x}$ which is closer to the root is thus iteratively called over and over again. When look at the calculation rule of the Newton method:
$$\mathbf{x}_{n+1} = \mathbf{x}_n - \mathbf{J}(\mathbf{x}_n)^{-1}\mathbf{f}(\mathbf{x}_n)$$
we see that we need to calculate the function $\mathbf{f}(\mathbf{x})$, the derivative of the function (the Jacobian $\mathbf{J}(\mathbf{x})$) and the inverse of the Jacobian matrix $\mathbf{J}(\mathbf{x})^{-1}$.

## Sneak Peak into Derivatives and Linear Equations

In Exercise 6. we will see how we can estimate the Jacobian of a function $\mathbf{f}(\mathbf{x})$ using different numeracial methods based on forward and backward differentiation. We can use forward diff to estimate the Jacobian of a function $\mathbf{f}(\mathbf{x})$ at a point $\mathbf{x}$:

In [ ]:
using ForwardDiff

f(x) = [x[1]^2 + x[2]^2 - 1, x[1] - x[2]^2]
q = [1.0, 1.0]
J_fd = ForwardDiff.jacobian(f, q)


In a following exercise we will see how we can solve a linear equation system:

In [ ]:
A = rand(3,3)
b = rand(3)
@show x = A\b # Simple version of solving a linear system

> Note: You should use the inverse function `inv(A)` to solve the linear equation. A more stable version of solving a linear system is based on LU factorization. You will learn this is a following exercise in more detail.

## Newton Method in Julia
You will learn that the inverse is not a good idea to use to solve the linear equation systems. Instead we can use the LU factorization to solve the linear equation system. We can first use the `lu` function to calculate the LU factorization of the Jacobian matrix and then use forward and backward substitution to solve $\mathbf{J}(\mathbf{x}_n)^{-1}\mathbf{f}(\mathbf{x}_n)$ instead of calculating the inverse of the Jacobian matrix. We need to calculate the function $\mathbf{f}(\mathbf{x})$ and the Jacobian matrix $\mathbf{J}(\mathbf{x}_n)$ in each iteration, since the Jacobian matrix depends on the point $\mathbf{x}_n$. 

First we define the function `J(x)` which calculates the Jacobian. We can use the `ForwardDiff.jl` package to calculate the Jacobian matrix of a function $\mathbf{f}(\mathbf{x})$ at a point $\mathbf{x}$:


In [ ]:
J(x) = ForwardDiff.jacobian(f, x)

Now we define a function `newton(f, J, x)` which implements the Newton method. Where `f` is the function $\mathbf{f}(\mathbf{x})$, `J` is the function `J(x)` which calculates the Jacobian matrix $\mathbf{J}(\mathbf{x})$ and `x` is the start value $\mathbf{x}_0$. The function `newton` returns the root of the function $\mathbf{f}(\mathbf{x})$. The function iteratively calls the calculation rule of the Newton method using the LU factorization until the desired accuracy is reached. The desired accuracy is defined by the parameter `tolerance` which is set to `1e-10` by default. The maximum number of iterations is defined by the parameter `maxiter` which is set to `1000` by default. 

In [ ]:
using LinearAlgebra

function newton(f, J, x, solver)
   h = Inf64
   tolerance = 10^(-10)
   iter = 0
   while (norm(h) > tolerance)
      iter += 1
      if iter > 1000
         error("Too many iterations")
      end
      h = solver(J(x), f(x))
      x = x - h
   end
   return x
end

In [ ]:
function simple_solver(A, b)
    return  # TODO: Implement a simple solver
end

Let's test the Newton method for the function $\mathbf{f}(\mathbf{x})$:
$$
\mathbf{f}(\mathbf{x}) = \begin{bmatrix}
  \rule[2mm]{0pt}{1em}x_1^2 + x_2^2 - 1\\[2mm]
  x_1 - x_2
\end{bmatrix}
$$

Try to find the two roots of the function $\mathbf{f}(\mathbf{x})$ by using the Newton method:

In [ ]:
f(x) = ...
x = [0.6, -1]
newton(f, J, x, simple_solver)

In [ ]:
x = [0.6, 1]
newton(f, J, x, simple_solver)

Let's try to find the two roots of another set of functions $\mathbf{f}(\mathbf{x})$:
$$
\mathbf{f}(\mathbf{x}) = \begin{bmatrix}
  \rule[2mm]{0pt}{1em}(1-x_1)^2 + (2-x_2)^2 - 5^2\\[2mm]
  (6-x_1)^2 + (1-x_2)^2 - 6.2^2\\[2mm]  
\end{bmatrix}
$$

In [ ]:
f(x) = ...
x = [1, -2]
newton(f, J, x, simple_solver)

In [ ]:
x = [3, 6]
newton(f, J, x, simple_solver)

# Solving the inverse kinematics of the Openmanipulator using the Newton method


First, let us define a position and heading we know we can reach with the OpenManipulator. We will use this as a starting point for the Newton method. We do this loading the kinematics library and estimating the forward kinematics of the OpenManipulator for the joint angles we know we can reach:

In [ ]:
goal_position, desired_orientation = OpenManipulatorKinematics.complete_forward_kinematics([0.6, 0.6, -0.5, -0.1])

To solve the inverse kinematics we first define a function that we want to solve. This function takes the joint angles as input and returns the difference between the current position/orientation and the desired position/orientation of the end effector. Our goal will be to find a point where this difference is zero. 

In [ ]:
using ForwardDiff

# Define a function to be solved for the root finding problem
function f_rootfindingProblem(q) 
    end_effector_position, end_effector_orientation = OpenManipulatorKinematics.complete_forward_kinematics(q)
    # TODO: generate a system of equations that will allow you to find the joint angles that will place the end effector at the goal position
    # use goal_position and end_effector_position to generate the system of equations
    # note that we have to apply a simple trick to the orientation to make it easier to solve the system of equations.
    # In other words you need to combine the orientation error into one single error equation
    f_1 = ...
    f_2 = ...
    f_3 = ...
    f_4 = ...
    return [f_1, f_2, f_3, f_4]
end

In [ ]:
function J_openManipulator(q)
    # Calculate the Jacobian matrix using ForwardDiff.jacobian
    J = ForwardDiff.jacobian(f_rootfindingProblem, q)
    return J
end

In [ ]:
q_init = [0.5, 0.5, -0.5, -0.1] # A position relatively close to the goal position
newton(f_rootfindingProblem, J_openManipulator, q_init, simple_solver)

What happens if we just initialize with $0$?

In [ ]:
q_init = [0.0, 0.0, 0.0, 0.0]
newton(f_rootfindingProblem, J_openManipulator, q_init, simple_solver)

Why did this not work? Let's look at the determinant of the Jacobian matrix. The determinant of the Jacobian matrix is a measure of how much the volume of the input space is scaled by the transformation. If the determinant is zero, the transformation is singular and the inverse kinematic problem has no solution. If it is close to zero the transformation is close to singular and the inverse kinematic problem is ill-conditioned. We can calculate the determinant of the Jacobian matrix using the `det` function.

In [ ]:
q_init = [0.0, 0.0, 0.0, 0.0]
jac_0 = J_openManipulator(q_init)
det(jac_0) 

This is bad. A linear equation with this Jacobian is ill-posed. We need to initialize with a value that is close to the solution and which results in a det of the Jacobian which is not too close to zero. Let's try to find a good starting point by trial and error:

In [ ]:
q_init = [0.0, 0.0, 0.0, 0.0] # TODO: Find a position that does result in a higher value of the determinant
jac_0 = J_openManipulator(q_init)
det(jac_0)

In [ ]:
newton(f_rootfindingProblem, J_openManipulator, q_init, simple_solver)

How good does this work if we start from some random joint angles?

In [ ]:
for i in 1:10
    q_init = rand(4)
    q = newton(f_rootfindingProblem, J_openManipulator, q_init, simple_solver)
    if norm(f_rootfindingProblem(q)) < 10^(-5)
        println("Found a solution, q = ", q)
    end
end

Ok, we see that we need to find a good starting point for the Newton method, otherwise we get a solution which is not reachable by the robot. One way could be to to check the joint angles if they are possible for the given position and heading of the the OpenManipulator. Could you think of other ways to find a good starting point for the Newton method or to improve the solutions given by the Newton method?


## Controling the OpenManipulator with the inverse kinematic

Intialize the Openmanipulator simulation: 

In [ ]:
using RigidBodyDynamics
using MeshCatMechanisms
using MeshCat
using LinearAlgebra
using StaticArrays

In [ ]:
srcdir = "../open_manipulator_description/urdf/"
urdf = joinpath(srcdir, "open_manipulator.urdf")
mechanism = parse_urdf(urdf)
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf));
render(mvis)

In [ ]:
state = mvis.state
set_configuration!(mvis, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
zero_velocity!(state)
@show joint_state = mvis.state.q
@show velocity = mvis.state.v
om_joints = joints(mechanism);

Setting the goal position and heading of the OpenManipulator:

In [ ]:
@show goal_angles = [0.5, 0.0, 0.0, 0.1]
goal_position, desired_orientation = OpenManipulatorKinematics.complete_forward_kinematics(goal_angles)
@show goal_position;

Let's estimate the joint angles using the inverse kinematics here so that we see if we find a solution

In [ ]:
q_init = [0.0, 0.0, 0.0, 0.0] # A position relatively close to the goal position
q_desired_inv_kinematic = newton(f_rootfindingProblem, J_openManipulator, q_init, simple_solver);
@show q_desired_inv_kinematic;

Implementing the control methods to move the OpenManipulator to the goal position and heading:

In [ ]:
function joint_position_control(q_state, v_state, q_desired)
    kp = 12.0
    v_damping = 0.1
    max_torque = 10.0

    torque = kp * (q_desired - q_state) - v_damping * v_state
    torque = clamp.(torque, -max_torque, max_torque)

    return torque
end;

In [ ]:
function simple_joint_control!(torques::AbstractVector, t, state::MechanismState)
    # add the last two joints to q_desired
    q_desired_all_joints = [q_desired_inv_kinematic; 0.0; 0.0]

    for j_iter in 1:6
        if j_iter > 4
            state.v[j_iter] = 0.0
        end
        torques[velocity_range(state, om_joints[j_iter])] .= joint_position_control(state.q[j_iter], state.v[j_iter], q_desired_all_joints[j_iter])
    end
end;

In [ ]:
set_configuration!(mvis, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
zero_velocity!(state)

In [ ]:
final_time = 2.0
ts, qs, vs = simulate(mvis.state, final_time, simple_joint_control!; Δt=1e-2);
MeshCatMechanisms.animate(mvis, ts, qs)
@show joint_state = mvis.state.q;

## Exercise: 

Play around with different goal positions. And see where problems occur. Can you think of a way to improve the control method?

In [ ]:
@show goal_angles = [0.0, 0.0, 0.0, 0.0] # TODO: test different goal angles or directly set goal positions 
goal_position, desired_orientation = OpenManipulatorKinematics.complete_forward_kinematics(goal_angles)
q_init = [0.0, 0.0, 0.0, 0.0] # A position relatively close to the goal position
q_desired_inv_kinematic = newton(f_rootfindingProblem, J_openManipulator, q_init, simple_solver);
@show q_desired_inv_kinematic;

set_configuration!(mvis, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
zero_velocity!(state)

In [ ]:
final_time = 2.0
ts, qs, vs = simulate(mvis.state, final_time, simple_joint_control!; Δt=1e-2);
MeshCatMechanisms.animate(mvis, ts, qs)
@show joint_state = mvis.state.q;